# Training Embeddings con GNN
Aca crearemos embeddings paa un grafo de Internet, es decir represenaciones de los SA a partir de la topologia y atributos de los SA.

Para esto tomamos 3 enfoques:


*   [Caso 1] Reconstruction Approach autoencoder: A traves de la prediccion de aristas construimos los embeddings de los nodos.
*   [Caso 1.1] Reconstruction Approach autoencoder: en ves de ocupar las bgp routes recolectadas de los ribs ocupamos las relaciones de AS rank y le damso attr a las aristas (es solo para probar que lo estemos ahciendo bien y las cosas tienen sentido)
*   [Caso 2]: Reconstruction Approach attribute Masking:
*   Caso 3: Task Generation Pre-calculated descriptor

Para cada uno crearemos un ejemplo.


In [ ]:
!pip install  dgl -f https://data.dgl.ai/wheels/torch-2.3/cu121/repo.html

In [32]:
%load_ext autoreload
%autoreload 2
# from modules.gnn import GNN
import scipy.sparse as sp
import numpy as np
import os

from modules.graph import Graph, create_files
from modules.gnn import GNN
from modules.gnn_models import GCN, GraphSAGE

import torch
import torch.nn.functional as F

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
# Definimos las rutas de los archivos
BASE_PATH = os.getcwd() + "/data/"
relationships_file = BASE_PATH + "CAIDA_AS_Relationships/Serial_1/20220701.as-rel.txt.bz2"
features_file = BASE_PATH + "/node_features.csv"
rib_path = BASE_PATH + 'sanitized_rib.txt'
dataset_graph_path = BASE_PATH + 'dgl_graph/'

## Creacion Grafo 
Creamos un grafo nx y dgl, ademas de los archivos edges.csv y nodes.csv a partir de archivos ribs previamente creados o de archivo CAIDA AS Relationships.

Crear esos archivos una unica vez con create_graph() una ves ya creados los archivos edges.csv y nodes.csv puedo ocupar directamente la funcion 

Se le puede indicar el maximo de bgp paths que se quiere (hehco para cuando se leen ribs no de caida) 

### CASO 1: RIBs
* Creacion de grafo a partir de paths recolectados de las RIBs por BGPStream
* Por ahora le asignamos a todos los nodos embeddings iniciales de de dimension 32 parte con puros 1s todos


In [34]:
# Definimos las listas de features

LIST_FEATURES_NO_CATEG = ['ASN', 'AS_rank_numberAsns', 'AS_rank_numberPrefixes', 'AS_rank_numberAddresses', 'AS_rank_total', 'AS_rank_customer',
                 'AS_rank_peer', 'AS_rank_provider', 'peeringDB_ix_count', 'peeringDB_fac_count', 'AS_hegemony', 'cti_top', 'cti_origin']

LIST_FEATURES_CATEG = ['AS_rank_continent',
                        'peeringDB_info_ratio',
                        'peeringDB_info_scope',
                        'peeringDB_info_type',
                        'peeringDB_policy_general'
                        'ASDB_C1L1']

list_feat = LIST_FEATURES_NO_CATEG + LIST_FEATURES_CATEG

features_file = 'node_degrees' #'' # 'node_degrees' # las features que se le agregaran seran 
type = 'MultiDiGraph'#"DiGraph" # MultiDiGraph
relationships_file
MAX_PATHS = 100000
graph_case1 = create_files(type, 
            dataset_graph_path,
            file = rib_path, 
            features_file = features_file, 
            from_caida=False, 
            remove_degree=3,
            debug=True,
            max_paths = MAX_PATHS)
 
print('[NX Graph]',graph_case1.nx_graph)

[CaRPETA CREADA]:  /home/valentina/Desktop/GIT/TrabajoTesis/data/dgl_graph/
[ARCHIVO EDGES.CSV CREADO]
[NX Graph]:  MultiDiGraph with 36173 nodes and 330327 edges
[TOPOLOGIA CREADA]
[SAVE IN: /home/valentina/Desktop/GIT/TrabajoTesis/data/dgl_graph/nodes.csv]
[FEATURES CREADOS]
[META CREADO]
[NX Graph]:  MultiDiGraph with 26185 nodes and 320339 edges
[NX Graph]:  MultiDiGraph with 26088 nodes and 320242 edges
[NX Graph]:  MultiDiGraph with 26088 nodes and 320242 edges
[NX Graph] MultiDiGraph with 26088 nodes and 320242 edges


### Caso 2
* Creacion del grafo a partir de CAIDA AS Relationships (AS Rank) 
* Se les da atributos a los edges correspondientes al tipo de relacion que comparten
* Es solo de prueba 

In [9]:
# Definimos las listas de features
LIST_FEATURES_NO_CATEG = ['ASN', 'AS_rank_numberAsns', 'AS_rank_numberPrefixes', 'AS_rank_numberAddresses', 'AS_rank_total', 'AS_rank_customer',
                 'AS_rank_peer', 'AS_rank_provider', 'peeringDB_ix_count', 'peeringDB_fac_count', 'AS_hegemony', 'cti_top', 'cti_origin']

LIST_FEATURES_CATEG = ['AS_rank_continent',
                        'peeringDB_info_ratio',
                        'peeringDB_info_scope',
                        'peeringDB_info_type',
                        'peeringDB_policy_general'
                        'ASDB_C1L1']

list_feat = LIST_FEATURES_NO_CATEG + LIST_FEATURES_CATEG

features_file =  'data/node_features.csv'  #'node_degrees' #'' # 'node_degrees' # las features que se le agregaran seran 
type = "DiGraph" # MultiDiGraph

max_paths = 100000
graph_case1 = create_files(type, 
             dataset_graph_path,
             relationships_file, 
             features_file, 
             from_caida=True, 
             remove_degree=3,
             debug=True,
max_paths = None)
 
print('[NX Graph]',graph_case1.nx_graph)

[CaRPETA CREADA]:  /home/valentina/Desktop/GIT/TrabajoTesis/data/dgl_graph/
[NX Graph]:  DiGraph with 74140 nodes and 379420 edges
[TOPOLOGIA CREADA]
[todos attr]
[bbbbbbbbbbbbbbbbbbbbbb]
[NX Graph]:  DiGraph with 74140 nodes and 379420 edges
[FEATURES CREADOS]
[META CREADO]
[NX Graph]:  DiGraph with 46714 nodes and 351994 edges
[NX Graph]:  DiGraph with 46237 nodes and 351517 edges
[NX Graph]:  DiGraph with 46229 nodes and 351509 edges
[NX Graph] DiGraph with 46229 nodes and 351509 edges


## Encode-Decoder 

In [ ]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.data import CoraGraphDataset
from dgl.dataloading import negative_sampler
import dgl.function as fn
from sklearn.metrics import roc_auc_score
from dgl.nn import SAGEConv, GraphConv, GATConv, GINConv, GatedGCNConv, GatedGraphConv

import numpy as np
import scipy.sparse as sp
from sklearn.metrics import roc_auc_score



from dgl.nn import SAGEConv
from dgl.nn import SAGEConv, GraphConv, GATConv, GINConv, GatedGCNConv, GatedGraphConv
import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools
import os



In [12]:
def compute_auc(pos_score,neg_score):
    scores = torch.cat([pos_score, neg_score]).numpy()
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).numpy()
    return roc_auc_score(labels, scores)


def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]
    )
    return F.binary_cross_entropy_with_logits(scores, labels)


### Link Prediction
Encoder : GNN
Decoder : DotProduct

In [13]:
gnn = GNN(debug=True)
gnn.load_dataset(dataset_graph_path, force_reload=True)

# FIXME: Cambiar, por mientras se estan agregando feat aleatorias o 1s a los nodos
# gnn.dgl_graph.ndata['feat'] = torch.ones(gnn.dgl_graph.num_nodes(), 128)            # features de tamano 128

print('[ATTR SHAPE]: ',gnn.dgl_graph.ndata['feat'].shape)

gnn.split_graph(train_size=0.8)

Done saving data into cached files.
Graph(num_nodes=74140, num_edges=351509,
      ndata_schemes={'feat': Scheme(shape=(72,), dtype=torch.float32)}
      edata_schemes={'Relationship': Scheme(shape=(), dtype=torch.int64)})
[ATTR SHAPE]:  torch.Size([74140, 72])
Generando 351509 aristas negativas...
Aristas negativas generadas: 351508


In [14]:

model = GraphSAGE(gnn.train_g.ndata["feat"].shape[1], 16,16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# ----------- 4. training -------------------------------- #

all_logits = []
for e in range(100):
    # forward
    model.train()
    h = model.encode(gnn.train_g, gnn.train_g.ndata["feat"])

    pos_score = model.decodeDotProduct(gnn.train_pos_g, h)
    neg_score = model.decodeDotProduct(gnn.train_neg_g, h)


    loss = compute_loss(pos_score, neg_score)

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if e % 5 == 0:
        print("In epoch {}, loss: {}".format(e, loss))

# ----------- 5. check results ------------------------ #
from sklearn.metrics import roc_auc_score

with torch.no_grad():
    pos_score = model.decodeDotProduct(gnn.test_pos_g, h)
    neg_score = model.decodeDotProduct(gnn.test_neg_g, h)
    print("AUC", compute_auc(pos_score, neg_score))


In epoch 0, loss: 1.8114488124847412
In epoch 5, loss: 0.5880826115608215
In epoch 10, loss: 0.5368225574493408
In epoch 15, loss: 0.5133905410766602
In epoch 20, loss: 0.5007092356681824
In epoch 25, loss: 0.48712238669395447
In epoch 30, loss: 0.47574564814567566
In epoch 35, loss: 0.46585574746131897
In epoch 40, loss: 0.4587332308292389
In epoch 45, loss: 0.45330360531806946
In epoch 50, loss: 0.4496583938598633
In epoch 55, loss: 0.44653990864753723
In epoch 60, loss: 0.44383367896080017
In epoch 65, loss: 0.4414408206939697
In epoch 70, loss: 0.439255952835083
In epoch 75, loss: 0.43731144070625305
In epoch 80, loss: 0.4355417788028717
In epoch 85, loss: 0.43387046456336975
In epoch 90, loss: 0.4323161542415619
In epoch 95, loss: 0.4308710992336273
AUC 0.9344419057524246


In [16]:
print(f'[MODELO] {model} ')
print(f'[EMBEDDINGS] {h} ')

[MODELO] GraphSAGE(
  (conv1): SAGEConv(
    (feat_drop): Dropout(p=0.0, inplace=False)
    (fc_neigh): Linear(in_features=72, out_features=16, bias=False)
    (fc_self): Linear(in_features=72, out_features=16, bias=True)
  )
  (conv2): SAGEConv(
    (feat_drop): Dropout(p=0.0, inplace=False)
    (fc_neigh): Linear(in_features=16, out_features=16, bias=False)
    (fc_self): Linear(in_features=16, out_features=16, bias=True)
  )
  (decoder): MLPPredictor(
    (W1): Linear(in_features=32, out_features=16, bias=True)
    (W2): Linear(in_features=16, out_features=1, bias=True)
  )
) 
[EMBEDDINGS] tensor([[ 0.0932,  0.1134,  0.0181,  ..., -0.1633,  0.1197, -0.2719],
        [ 0.1709,  0.0605,  0.0149,  ..., -0.1122,  0.0548, -0.1721],
        [ 0.4162,  0.0903,  0.0248,  ..., -0.4585,  0.0986, -0.5992],
        ...,
        [ 0.0691,  0.0790, -0.0193,  ..., -0.1449,  0.0954, -0.2131],
        [ 0.0502, -0.0102, -0.0521,  ..., -0.0742,  0.0468, -0.0872],
        [-0.2538,  0.0183,  0.0726,  

In [28]:
# Guardar modelo y embeddings 
# Guaradar el modelo
torch.save(model.state_dict(), f'data/model_emb.pth')

# Guardar los embeddings
torch.save(h, f"data/embeddings.pt")


### Link Prediction
Encoder : GNN
Decoder : MLP

In [10]:
gnn = GNN(debug=True)
gnn.load_dataset(dataset_graph_path, force_reload=True)

# FIXME: Cambiar, por mientras se estan agregando feat aleatorias o 1s a los nodos
# gnn.dgl_graph.ndata['feat'] = torch.ones(gnn.dgl_graph.num_nodes(), 128)            # features de tamano 128

print('[ATTR SHAPE]: ',gnn.dgl_graph.ndata['feat'].shape)

gnn.split_graph(train_size=0.8)


Done saving data into cached files.
Graph(num_nodes=36173, num_edges=320242,
      ndata_schemes={'feat': Scheme(shape=(2,), dtype=torch.float32)}
      edata_schemes={'Relationship': Scheme(shape=(), dtype=torch.float32)})
[ATTR SHAPE]:  torch.Size([36173, 2])
Generando 320242 aristas negativas...
Aristas negativas generadas: 320241


In [11]:
model = GraphSAGE(gnn.train_g.ndata["feat"].shape[1], 16,16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# ----------- 4. training -------------------------------- #

all_logits = []
for e in range(100):
    # forward
    model.train()
    h = model.encode(gnn.train_g, gnn.train_g.ndata["feat"])

    pos_score = model.decodeMLP(gnn.train_pos_g, h)
    neg_score = model.decodeMLP(gnn.train_neg_g, h)


    loss = compute_loss(pos_score, neg_score)

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if e % 5 == 0:
        print("In epoch {}, loss: {}".format(e, loss))

# ----------- 5. check results ------------------------ #
from sklearn.metrics import roc_auc_score

with torch.no_grad():
    pos_score = model.decodeMLP(gnn.test_pos_g, h)
    neg_score = model.decodeMLP(gnn.test_neg_g, h)
    print("AUC", compute_auc(pos_score, neg_score))

In epoch 0, loss: 0.7014414072036743
In epoch 5, loss: 0.6320050954818726
In epoch 10, loss: 0.4730013310909271
In epoch 15, loss: 0.250716894865036
In epoch 20, loss: 0.13942362368106842
In epoch 25, loss: 0.11567464470863342
In epoch 30, loss: 0.1040361300110817
In epoch 35, loss: 0.09642818570137024
In epoch 40, loss: 0.09165021777153015
In epoch 45, loss: 0.08361601829528809
In epoch 50, loss: 0.07863310724496841
In epoch 55, loss: 0.07742946594953537
In epoch 60, loss: 0.07598143815994263
In epoch 65, loss: 0.07474548369646072
In epoch 70, loss: 0.07381406426429749
In epoch 75, loss: 0.07291460037231445
In epoch 80, loss: 0.07191368192434311
In epoch 85, loss: 0.07128756493330002
In epoch 90, loss: 0.070730060338974
In epoch 95, loss: 0.07016365975141525
AUC 0.9950219820341152


In [12]:
# Se incluyen los parametros tanto de MLP como de la GNN

for name, param in model.named_parameters():
    print(name, param.shape)


conv1.fc_neigh.weight torch.Size([16, 2])
conv1.fc_self.weight torch.Size([16, 2])
conv1.fc_self.bias torch.Size([16])
conv2.fc_neigh.weight torch.Size([16, 16])
conv2.fc_self.weight torch.Size([16, 16])
conv2.fc_self.bias torch.Size([16])
decoder.W1.weight torch.Size([16, 32])
decoder.W1.bias torch.Size([16])
decoder.W2.weight torch.Size([1, 16])
decoder.W2.bias torch.Size([1])


#### Guardar Modelo y Embeddings

In [17]:
# Guaradar el modelo
torch.save(model.state_dict(), 'data/model_emb.pth')

# Guardar los embeddings
torch.save(h, "data/embeddings.pt")


In [ ]:
# import pandas as pd

# # Guaradar el modelo
# torch.save(model.state_dict(), 'model.pth')

# # Guardar los embeddings de los nodos después del entrenamiento
# with torch.no_grad():
#     # Calcular los embeddings finales
#     # final_embeddings = model.encode(train_g, train_g.ndata["feat"]).detach().cpu().numpy()
#     final_embeddings = h.detach().cpu().numpy()

#     # Crear un DataFrame para guardar los embeddings
#     # Obtener ids de los nodos (ASN)
#     # node_ids = train_g.nodes().numpy()
#     node_ids = gnn.dgl_graph.nodes().numpy()

#     emb_df = pd.DataFrame(final_embeddings, index=node_ids)
#     emb_df.index.name = "node_id"

#     # Guardar en un archivo CSV
#     emb_df.to_csv("node_embeddings.csv")
#     np.save("node_embeddings.npy", final_embeddings)
#     print("Embeddings guardados en 'node_embeddings.csv'")

# emb_df

Embeddings guardados en 'node_embeddings.csv'


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
node_id,,,,,,,,,,,,,,,,
0,1.101930,-0.335221,-3.074711,2.228754,-0.627656,-2.087168,2.749624,-0.108119,-1.696507,3.432124,-1.217278,-3.109729,2.090653,-2.463471,3.261267,-0.414159
1,1.723648,-1.042069,-2.855011,1.761993,-0.334552,-1.730516,3.329224,-0.171126,-2.141119,3.345304,-0.868315,-3.146378,2.222488,-2.328668,3.026546,-0.028456
2,1.158578,-1.128411,-3.201206,2.089556,-0.243688,-2.558243,3.160474,1.796038,-1.859045,2.827035,-1.029408,-3.751191,1.974931,-3.294911,3.833996,-0.531082
3,1.077238,-0.910498,-3.221866,2.284746,-0.425296,-2.518895,3.067572,1.671393,-1.717550,2.932030,-1.126791,-3.613750,2.083611,-3.193594,3.809445,-0.619963
4,0.975382,-0.805138,-3.114366,2.234302,-0.293983,-2.377515,2.980376,1.467021,-1.709631,2.908147,-1.105135,-3.563770,2.021119,-3.159290,3.726334,-0.601326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36168,0.860783,-1.317075,-3.237432,2.151350,0.031120,-2.795558,3.234994,3.156262,-1.719627,2.397869,-1.008901,-4.016392,1.883370,-3.971356,4.242860,-0.700788
36169,-0.368163,1.001421,-1.172179,2.207545,-1.146080,-1.596958,0.281243,-0.243500,1.134093,1.264028,-0.823900,-1.051718,-0.904146,-1.968068,1.897566,-1.251757
36170,1.198884,-0.767427,-3.267465,2.391738,-0.707687,-2.478895,3.019535,1.173728,-1.692087,3.166048,-1.204719,-3.448300,2.203176,-2.884263,3.673004,-0.615303


In [ ]:
# # Cargar embeddings
# embeddings = np.load("node_embeddings.npy")
# print("[SHAPE]",embeddings.shape)
# print("[EMBEDDING]",embeddings[0])

[SHAPE] (36173, 16)
[EMBEDDING] [ 1.1019298 -0.3352208 -3.0747113  2.2287536 -0.6276562 -2.0871685
  2.7496235 -0.1081185 -1.6965067  3.4321244 -1.2172776 -3.1097293
  2.0906534 -2.4634714  3.2612672 -0.4141587]


Cosas a tener consideraciom:
- GNN al crear un grafo, crea tdoos los nodos que se encuentran en el rango entregado. De esta forma pueden quedar nodos aislados, sin embargo ento causa que no se puedan ir 'actualizando' pero al agregar self_loop se actualizan con si mismos y supondremos que con ello algun patron para este tipo de nodos.
- La entrega de embeddings finales esta en orden el ASN de valor inferios hasta el ASN de valor maximo.
- Lo guardamso en un .csv dondde se asocia 'node_id' con su embeddings y donde node_id es el ASN del Sistema Autonomo. 

In [ ]:
# gnn.train_g.ndata['feat'] 

tensor([[0.2114, 0.0000],
        [0.2993, 0.2341],
        [0.2607, 0.0000],
        ...,
        [0.1117, 0.0000],
        [0.1117, 0.0000],
        [0.1117, 0.0000]])

# DeepWalk (2014)

DeepWalk como en otros metodos de NLP donde la frecuencia de palabras tiene un comportamiento aque siguie una distribución de ley de potencia (Power Law Distribution). (Pocas palabras tienen una alta aparicion mientras que la mayoria raramente aparece).
Este siguie un comportamiento similarar, donde en caminatas aleatorias son pocos los nodos que aparecen frecuentemente, mientras que la mayoria no.

* Se apoyan en el mismo concepto que plantea word2vec
* De igual manera que word2vec busca estimar la probabilidad de ocurrencia de una palabra en una frase, dadas las palabras cercanas

Algoritmo:

* DeepWalk genera caminos aleatorios recorriendo el grafo múltiples veces desde todos los nodos con una longitud determinada.
* Deep Walk se les llama caminos aleatorios de primer orden ya que se basan en probabilidades de transición de nodo-a-nodo;


Resultados esperados:
* Bajo la homophily hypothesis, nodos muy conectados entre sí y que pertenecen a las mismas comunidades en el grafo, deben producir embeddings cercanos. 
* Bajo la structural hypothesis, nodos cuya función estructural en el grafo es similar (por ejemplo, nodos que actúan como hub) también deben producir embeddings cercanos.

DeepWalk es un algoritmo para aprender embeddings de nodos en un grafo.
¿Cómo funciona?

    Hace caminatas aleatorias (random walks) por el grafo, partiendo de cada nodo, como si fueran frases.

    Cada caminata es una secuencia de nodos, como una frase es una secuencia de palabras.

    Usa Word2Vec (Skip-gram) para aprender embeddings de nodos, tratándolos como si fueran palabras.

        Nodo = palabra

        Caminata = frase

        Grafo = corpus

In [35]:
from dgl.nn import DeepWalk
from torch.utils.data import DataLoader
from torch.optim import SparseAdam

In [36]:
gnn = GNN(debug=True)
gnn.load_dataset(dataset_graph_path, force_reload=True)

# FIXME: Cambiar, por mientras se estan agregando feat aleatorias o 1s a los nodos
# gnn.dgl_graph.ndata['feat'] = torch.ones(gnn.dgl_graph.num_nodes(), 128)            # features de tamano 128

print('[ATTR SHAPE]: ',gnn.dgl_graph.ndata['feat'].shape)

gnn.split_graph(train_size=0.8)


Done saving data into cached files.
Graph(num_nodes=36173, num_edges=320242,
      ndata_schemes={'feat': Scheme(shape=(2,), dtype=torch.float32)}
      edata_schemes={'Relationship': Scheme(shape=(), dtype=torch.float32)})
[ATTR SHAPE]:  torch.Size([36173, 2])
Generando 320242 aristas negativas...
Aristas negativas generadas: 320241


In [52]:
emb = DeepWalk(gnn.dgl_graph,
               emb_dim=32,        # tamaño del embedding
               walk_length=6,     # número de saltos por walk
                    # tamaño de la ventana de contexto
            #    walks_per_node=10,  # número de walks por nodo
            #    num_negative=5
            # número de negativos por positivo
)

In [55]:
from torch.utils.data import DataLoader

In [68]:
emb = DeepWalk(gnn.dgl_graph)
dataloader = DataLoader(torch.arange(gnn.dgl_graph.num_nodes()), 
                        batch_size=128,
                        shuffle=True, 
                        collate_fn=emb.sample)

optimizer = SparseAdam(emb.parameters(), lr=0.01)

num_epochs = 1

for epoch in range(num_epochs):
    for batch_walk in dataloader:
        print("batch_walk max:", batch_walk.max().item())
        print("embedding size:", emb.node_embed.num_embeddings)

        print("[BATCH]",batch_walk.shape )
        loss = emb(batch_walk)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


batch_walk max: 35760
embedding size: 36173
[BATCH] torch.Size([128, 40])


IndexError: index out of range in self

In [44]:

dataloader = DataLoader(torch.arange(gnn.dgl_graph.num_nodes()),
                        batch_size=500,
                        shuffle=True, 
                        collate_fn=emb.sample,)

optimizer = SparseAdam(emb.parameters(), lr=0.01)

num_epochs = 1

for epoch in range(num_epochs):
    for batch_walk in dataloader:
        # print("[BATCH]",batch_walk )
        loss = emb(batch_walk)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


# for epoch in range(num_epochs):
#     for batch_walk in dataloader:
#         # Reemplazar -1 por un nodo válido (ej: nodo 0)
#         batch_walk[batch_walk == -1] = 0

#         loss = emb(batch_walk)
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
        


IndexError: index out of range in self

In [11]:
h = emb.node_embed.weight.detach()
print("SHAPE",h.shape)
print("EMBEDDINGS",h)

SHAPE torch.Size([36173, 32])
EMBEDDINGS tensor([[ 0.1550, -0.3473,  0.4126,  ...,  0.3885,  0.3698,  0.2183],
        [ 0.2898, -0.3493,  0.3488,  ...,  0.3034,  0.3284,  0.3131],
        [ 0.3237, -0.3049,  0.3346,  ...,  0.3572,  0.3408,  0.3349],
        ...,
        [ 0.0934, -0.1191,  0.1167,  ...,  0.1023,  0.0657,  0.1067],
        [ 0.0699, -0.0926,  0.1158,  ...,  0.1232,  0.0960,  0.0738],
        [ 0.1134, -0.1234,  0.1093,  ...,  0.0711,  0.0880,  0.0889]])


In [12]:
# Guardar los embeddings
torch.save(h, "data/embeddings.pt")

# Node2Vec (2016)

* Surge con el fin de generalizar DeepWalk.
* Node2Vec emplea caminos aleatorios de segundo orden, pues se basa en probabilidades de transición de nodo-a-nodo-a-nodo (o vértice-a-vértice). Es decir, avanzar hacia un nodo determinado no viene sólo dado por el nodo en el que nos encontremos, sino también por el nodo anterior de donde vengamos.

* Se apoyan en el mismo concepto que plantea word2vec
* De igual manera que word2vec busca estimar la probabilidad de ocurrencia de una palabra en una frase, dadas las palabras cercanas

Node2Vec es una mejora de DeepWalk. También genera caminatas y usa Word2Vec, pero:
    Introduce dos parámetros:
        pp: controla la probabilidad de volver atrás (tipo DFS).

        qq: controla la probabilidad de explorar localmente (tipo BFS).

    Así, Node2Vec puede balancear entre exploración profunda (estructural) y local (de vecindad).

Esto permite obtener embeddings que capturan diferentes tipos de relaciones:

    Parecidos en rol (ej: puentes entre comunidades).

    Parecidos en vecindad (ej: miembros de la misma comunidad).